In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'data/ds_salaries.csv')

#### Preprocesamiento

In [3]:
df.head(3)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S


In [4]:
df.columns #saber columnas

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary', 'salary_currency', 'salary_in_usd', 'employee_residence',
       'remote_ratio', 'company_location', 'company_size'],
      dtype='object')

In [5]:
df.shape #saber las dimensiones

(3755, 11)

In [6]:
df.dtypes #saber los tipos de datos

work_year              int64
experience_level      object
employment_type       object
job_title             object
salary                 int64
salary_currency       object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object

In [7]:
df.isnull().sum() #saber los valores nulos

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [8]:
df.duplicated().sum() #saber los valores duplicados

np.int64(1171)

In [9]:
duplicados=pd.DataFrame(df.value_counts())
duplicados[duplicados['count']>1]

count
work_year experience_level employment_type job_title            salary salary_currency salary_in_usd employee_residence remote_ratio company_location company_size       
2022      SE               FT              Data Scientist       191475 USD             191475        US                 100          US               M                21
                                                                141525 USD             141525        US                 100          US               M                21
2023      SE               FT              Data Engineer        252000 USD             252000        US                 0            US               M                13
                                                                129000 USD             129000        US                 0            US               M                13
2022      SE               FT              Data Engineer        130000 USD             130000        US                 0            US               M                12
...                                                                                                                                                                   ...
                                           Data Scientist       119300 USD             119300        US                 0            US               L                 2
2023      SE               FT              Data Quality Analyst 100000 USD             100000        US                 0            US               M                 2
                                                                80000  USD             80000         US                 0            US               M                 2
2022      EN               FT              Data Analyst         50000  USD             50000         US                 50           US               L                 2
2023      MI               FT              Data Analyst         85000  USD             85000         US                 0            US               M                 2

[544 rows x 1 columns]

In [10]:
df.drop_duplicates(inplace=True) #eliminar duplicados

In [11]:
df.shape

(2584, 11)

### EDA

In [12]:
df.head(1)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L


en que pais hay compañias mas grandes?

In [13]:
pais_df1, cantidad_df1 = df.loc[df['company_size'] == 'L', 'company_location'].value_counts().idxmax(), df.loc[df['company_size'] == 'L', 'company_location'].value_counts().max()

pais_df1, cantidad_df1




('US', np.int64(220))

En que puesto se gana mas

In [14]:
df1, df2 = df.groupby('job_title')['salary_in_usd'].max().idxmax(), df.groupby('job_title')['salary_in_usd'].max().max()
df1, df2


('Research Scientist', np.int64(450000))

Tipo de trabajo, pais de residencia donde mas se cobra?

In [53]:
df3 = df.groupby(['job_title', 'company_location'])['salary_in_usd'].max().reset_index()
df3


,job_title,company_location,salary_in_usd
0,3D Computer Vision Researcher,AL,10000
1,3D Computer Vision Researcher,AS,20000
2,3D Computer Vision Researcher,CR,50000
3,3D Computer Vision Researcher,IN,5409
4,AI Developer,BA,120000
...,...,...,...
351,Research Scientist,US,450000
352,Software Data Engineer,AU,50000
353,Software Data Engineer,SG,75020
354,Staff Data Analyst,CA,15000


¿El nivel de  experiencia es proporcional al salario?

cuanto se cobra de media por cada nivel de experiencia?

In [52]:
df5 = df.groupby('experience_level')['salary_in_usd'].mean().reset_index()
df5

,experience_level,salary_in_usd
0,EN,72648.685185
1,EX,191078.208333
2,MI,101828.783133
3,SE,153897.435650


en qué tamaño de compañía trabajan más

por que porcentage promedio se aumenta el salario entre nivel de experiencia

en que pais se cobra  mas por tipo de empleo?

In [16]:
df.groupby(['company_location','job_title']).agg({'salary_in_usd':'mean'}).round(2).sort_values(by=['salary_in_usd','company_location'],ascending=False)

salary_in_usd
company_location job_title                                   
IL               AI Scientist                        423834.0
US               Data Analytics Lead                 405000.0
                 Data Science Tech Lead              375000.0
IN               AI Developer                        300000.0
US               Director of Data Science            294375.0
...                                                       ...
CH               Big Data Engineer                     5882.0
IN               Data Science Consultant               5707.0
                 3D Computer Vision Researcher         5409.0
                 Power BI Developer                    5409.0
CZ               NLP Engineer                          5132.0

[356 rows x 1 columns]

nivel de experiencia mas requerido

In [70]:
df.groupby(['job_title','experience_level']).size()


job_title                      experience_level
3D Computer Vision Researcher  EN                   2
                               MI                   1
                               SE                   1
AI Developer                   EN                   6
                               MI                   2
                                                   ..
Research Scientist             SE                  34
Software Data Engineer         MI                   1
                               SE                   1
Staff Data Analyst             EX                   1
Staff Data Scientist           SE                   1
Length: 192, dtype: int64

cuáles se repiten más en este set de datos, las empresas medianas, grandes o pequeñas

In [84]:
total_size_per_location = df.groupby('company_location')['company_size'].sum()
location_with_largest_size = total_size_per_location.idxmax()
total_size = total_size_per_location.max()

print(f'La ubicación con el mayor tamaño de compañía es: {location_with_largest_size} con un tamaño total de {total_size}.')

La ubicación con el mayor tamaño de compañía es: JP con un tamaño total de SSSSSS.
